## Pong 01

El siguiente programa muestra la primera versión de pong: 
Hemos definido una clae para la bola (`Ball`). La clase implementa los
métodos `update` (Por ahora solo se mueve hacia la derecha) y
`draw`, de forma que la responsabilidad de actualizar su posición
y la de pintarse en pantalla son asumidas por la propia bola:

In [5]:
# Version 1.1

import math
import pygame
import random
import time

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego

class Ball:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        self.speed = 10
        
    def update(self):
        self.x += self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.x-5, self.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)
        
pygame.init()
try:
    pygame.display.set_caption("Pong 01")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    ball = Ball(limits.centerx, limits.centery)
    clock = pygame.time.Clock()    
    in_game = True
    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        ball.update()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        ball.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

### Vamos a cambiar el código para que se mueva en diagonal

El problema es que si cambiamos el código del update, por ejemplo, a:
    
```Python
    class Ball:
        ...
        def update(self):
            self.x += self.speed
            self.y += self.speed
        ...
```

La velocidad de la bola no será la misma que cuando se movia en linea recta, porque 
ahora la bola no se desplaza 10 pixels cada frame; se desplaza más. Podemos calcular cuanto más por el [Teorema de Pitagoras](https://es.wikipedia.org/wiki/Teorema_de_Pit%C3%A1goras):
    
> Teorema de Pitágoras.- En un triángulo rectángulo, el cuadrado de la hipotenusa es igual a la suma de los cuadrados de los catetos.

Es decir, la distacia que viaja la pelota cada frame es igual a:

$$ d = \sqrt{10^2 + 10^2} = \sqrt{200} = 10\sqrt{2} \approx 14.142135623730951$$

Como podemos calcular facilmente usando la funcion `sqrt` (raíz cuadrada) del módulo `math`.

In [15]:
10 * math.sqrt(2)

14.142135623730951

![velocidad](velocidad.png)

Estos problemas y otros los resolvemos usando vectores. Básicamente, un vector
es simplemente un serie de números. Un vector de 2 dimensiones, por tanto, es una tupla
de dos numeros. Un vector de tres dimensione, una tripleta.

Con vectores podemos representar muchas cosas: La posición de un objeto, su velocidad, las fuerzas que actuan sobre el, etc... En el módulo [vectores.py](vectores.py) hemos definido una clase para trabajar con vectores de 2 dimensiones, llamada `Vector2`.

In [2]:
from vectores import Vector2

v = Vector2(3, 4)
print(v)

Vector2(3.0, 4.0)


Por convenio, y dado que los vectores se usan principalmente para trabajar 
en el espacio, se usan las letras $x$ e $y$ para representar el primer número
y el segundo, respectivamente, para el caso de los vectores de dos dimensiones. La clase `Vector2` también tiene esta capacidad:

In [6]:
v = Vector2(3, 4)
print('x:', v.x)
print('y:', v.y)

x: 3.0
y: 4.0


Una ventaja de usar vectores en vez de las tuplas de Python es que podemos definir
como queremos que se hagan las operaciones, por ejemplo, de suma. Si intentamos sumar
dos tuplas de dos elementos, el resultado es una tupla de 4 elementos:

In [4]:
t1 = (1, 2)
t2 = (3, 4)
t1 + t2

(1, 2, 3, 4)

En el espacio vectorial, la suma de vectores se define de otra manera, la suma
de ls vectores <1, 2> y <3, 4> debería dar como resultado un nuevo vector
bidimensional <4, 6>. Pyton tiene una característica de la programación
orientada a objetos llamada **sobrecarga de operadores**, que no es mas que una forma
abreviada de decir que podemos definir como se comportan determinados operadores
en nuestras clases.m

En el caso de Vectmor2, se ha **redefinido** o **sobrecargado** el operador `+`, de forma que ahora nuestros vectores se pueden sumar y el resultado es el que queremos:

In [10]:
t1 = Vector2(1, 2)
t2 = Vector2(3, 4)
print(t1 + t2)

Vector2(4.0, 6.0)


Una característica importante de los vectores es el concepto de **módulo**, que es
la distancia euclidea desde el cero de las coordenadas del vector. Es decir, si consideramos
el vector como una flecha que apunta desde el origen de coordenadas hasta un determinado
punto, el módulo sería la distancia de la flecha. La clase `Vector2` puede acceder (pero no modificar) al valor del móduo usando elcampo `mod`.

In [14]:
t = Vector2(3, 4)
print(t.mod)

5.0


El resultado es 5 porque, como vimos por el Teorema de Pitágoras:

$$ \sqrt{3^2+4^2} = \sqrt{9+15} = \sqrt{25} = 5 $$

Otro concepto importante, pero sencillo, es el de **vector unitario**: Un vector unitario
es cualquier vector cuyo módulo sea $1$.

Los vectores unitarios se suelen usar cuando nos interesa solo indicar una dirección, sin
que el módulo o la _longitud_ del vector sea relevante.

## Cambiamos la reprsentación de la posición de la pelota por un vector

En el siguiente código, cambiamos las propiedades `x` e `y` por un solo valor
de tipo `Vector2`, que llamaremos `pos`, y un vector unitario para representar
la dirección en la que tiene que moverse la pelota, que llamaremos `orientation`.
Usaremos un vector unitario porque solo nos interesa la dirección general en la
que se debe mover la pelota, la velocidad seguirá estando en el campo `speed`:

In [8]:
# Version 1.2

import math
import pygame
import random
import time

from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego

class Ball:
    
    def __init__(self, x=0, y=0):
        self.pos = Vector2(x, y)
        self.speed = 10
        self.orientation = Vector2(1, 0)
        
    def update(self):
        self.pos += self.orientation * self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.pos.x-5, self.pos.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)
        
pygame.init()
try:
    pygame.display.set_caption("Pong 01")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    ball = Ball(WIDTH//2, HEIGHT//2)
    clock = pygame.time.Clock()    
    in_game = True
    while in_game:
        for event in pygame.event.get():  # Obtener entradas
            if event.type == pygame.QUIT:
                in_game = False
        ball.update() # Recalcular el estado del juego
        screen.fill(BLACK) # Representamos el nuevo estado
        ball.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

Si ejecutamos el código anterior, veremos que se comporta igual. ¿Que hemos
ganado al representar la velocidad y la dirección con vectores? Pues
en primer lugar, hemos conseguido que el movimiento de la pelota
sea consistente, es decir, qe sea siempre de 10 pixels por frame (o el
valor que tenga el campo `speed`).

Para comprobarlo, veamos primero otra característica de los vectores. Si consideramos
el vector como una flecha en el espacio, con origen en el centro de coordenadas
y con la punta situada en las coordenadas del vector, esta flecha forma un ángulo
con los ejes de coordenadas. en concreto, el ángulo formado por el vector y el eje
positivo x se suele reprsentar con el símbolo $\theta$ o _theta_.

La clase `Vector2` tiene un atributo `theta` que es precisamente este ángulo, 
pero expresado en [**radianes**](https://es.wikipedia.org/wiki/Radi%C3%A1n).

In [20]:
v = Vector2(6, 3)
print(v.theta)

0.4636476090008061


Para pasar de radianes a grados, se usa la formula:
    $$ 2 \times \pi \times rad = 360º $$

O sea, que un radián equivale a $180/\pi$ grados. Si, por ejemplo, creamos un vector
unitario <1, 0>, el ángulo $\theta$ será cero, porque coincide con la horizontal y además
apunta hacia el lado positivo.

In [3]:
v = Vector2(1,0)
print(v.theta)

0.0


pero si ahora lo giramos ese vector, modificando $\theta$ para que equivalga a
45 grados (es decir, $\frac{\pi}{4}$), veremos que las componentes $x$ e $y$ del vector valen cada una $\sqrt{\frac12}$ para que su modulo pueda seguir valiendo uno, es decir, que
siga siendo un vector unitario:

In [4]:
v = Vector2(1,0)
v.theta = math.pi / 4
print(v.x, v.y, v.mod)

0.707106781 0.707106781 1.0


Asi que ahora podemos hacer que la pelota salga con el ángulo
que queramos de forma fácil, solo hay que modificar el campo
`orientation`, de forma que, justo despues de haberlo inicializado
al vector <1, 0>, modificar el valor de $\theta$ en un rango arbitrario.
El siguiente programa solo añade una línea a la versión 1.2:

In [11]:
# Version 1.3

import math
import pygame
import random
import time

from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego

class Ball:
    
    def __init__(self, x=0, y=0):
        self.pos = Vector2(x, y)
        self.speed = 10
        self.orientation = Vector2(1, 0)
        self.orientation.theta += random.uniform(-math.pi/4, math.pi/4)
        
    def update(self):
        self.pos += self.orientation * self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.pos.x-5, self.pos.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)
        
pygame.init()
try:
    pygame.display.set_caption("Pong 01")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    ball = Ball(WIDTH//2, HEIGHT//2)
    clock = pygame.time.Clock()    
    in_game = True
    while in_game: 
        for event in pygame.event.get(): # Obtener entradas
            if event.type == pygame.QUIT:
                in_game = False
        
        ball.update()       # Recalcular estado del juego
        screen.fill(BLACK)  # Representamos el nuevo estado
        ball.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()